# CHL Curve Conrady Prediction

This notebook uses the 3-term Conrady model to predict the longitudinal chromatic aberration (CHL) curve based on user-defined optical and sampling conditions. The `achromatcfw.core.cfw` module is then used for real-time evaluation of color fringes through focus.

In [1]:
# Add ../src to the Python search path
import sys
from pathlib import Path
sys.path.append(str(Path('..').resolve() / 'src'))
from chromf.cfw import fringe_width
from glass_map import glass_conrady, glass_sellmeier, glass_acf

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, FloatSlider, interact, Output
from IPython.display import display, clear_output


## 📐 Mathematical Model: Longitudinal Chromatic Focal Shift

This widget computes the **longitudinal chromatic shift curve** $ \Delta z(\lambda) $ for a thin achromatic doublet using Conrady dispersion and paraxial lens theory. The derivation incorporates user-defined glass types and anchor wavelengths.

---

### 📌 Notation

| Symbol                   | Units                     | Meaning |
|--------------------------|----------------------------|---------|
| $ \lambda $            | nm                        | Wavelength |
| $ \lambda_0 $          | nm                        | Design wavelength (**λ₀** slider) |
| $ \lambda_1,\,\lambda_2 $ | nm                    | Achromat anchor wavelengths (**λ₁**, **λ₂**) |
| $ f_0 $                | mm                        | Design focal length (**f₀** slider) |
| $ \Phi_{0,\mathrm{req}} = 1/f_0 $ | $mm^{-1}$     | Required system power at $ \lambda_0 $ |
| $ a_i,\ b_i,\ c_i $    | –, nm, $nm^{3.5}$        | Conrady coefficients for glass $ i = 1, 2 $ |
| $ K_i $                | $mm^{-1}$               | Shape factor of element $ i $ |
| $ \Phi_0,\ \alpha,\ \beta $ | mm$^{-1}$, mm$^{-1}$·nm, mm$^{-1}$·nm$^{3.5}$ | Power expansion coefficients |
| $ A,\ B,\ C $          | µm, µm·nm, µm·nm$^{3.5}$ | Longitudinal shift coefficients |
| $ \Delta z(\lambda) $  | µm                        | Axial shift from design focus |

---

### 🧮 Governing Equations

1. **Dispersion model (Conrady)**  
   $$
   n_i(\lambda) = 1 + a_i + \frac{b_i}{\lambda} + \frac{c_i}{\lambda^{3.5}}
   $$

2. **Element power**  
   $$
   \Phi_i(\lambda) = \bigl[n_i(\lambda) - 1\bigr] \cdot K_i
   $$

3. **Total system power**  
   $$
   \Phi(\lambda) = 
   (a_1K_1 + a_2K_2) + \frac{b_1K_1 + b_2K_2}{\lambda} + \frac{c_1K_1 + c_2K_2}{\lambda^{3.5}} 
   = \Phi_0 + \frac{\alpha}{\lambda} + \frac{\beta}{\lambda^{3.5}}
   $$

4. **Achromatism condition (anchor wavelengths)**  
   $$
   r = \frac{K_2}{K_1} = -\frac{b_1 \Delta\nu_{rb} + c_1 \Delta\nu_{rb,3.5}}{b_2 \Delta\nu_{rb} + c_2 \Delta\nu_{rb,3.5}}
   $$
   where  
   $$
   \Delta\nu_{rb} = \frac{1}{\lambda_1} - \frac{1}{\lambda_2}, \quad
   \Delta\nu_{rb,3.5} = \frac{1}{\lambda_1^{3.5}} - \frac{1}{\lambda_2^{3.5}}
   $$

5. **Power constraint at $ \lambda_0 $**  
   $$
   K_1 = \frac{\Phi_{0,\mathrm{req}}}
   {a_1 + r a_2 + \dfrac{b_1 + r b_2}{\lambda_0} + \dfrac{c_1 + r c_2}{\lambda_0^{3.5}}}, \qquad K_2 = r K_1
   $$

6. **Conversion to longitudinal shift coefficients**  
   $$
   A = \frac{1}{\Phi_0}, \qquad
   B = -\frac{\alpha}{\Phi_0^2}, \qquad
   C = -\frac{\beta}{\Phi_0^2}
   $$

7. **Longitudinal chromatic focal shift**  
   $$
   \Delta z(\lambda) = 
   \left( A + \frac{B}{\lambda} + \frac{C}{\lambda^{3.5}} \right)
   - \left( A + \frac{B}{\lambda_0} + \frac{C}{\lambda_0^{3.5}} \right)
   = B\,\Delta\nu + C\,\Delta\nu_{3.5}
   $$
   with  
   $$
   \Delta\nu = \frac{1}{\lambda} - \frac{1}{\lambda_0}, \quad
   \Delta\nu_{3.5} = \frac{1}{\lambda^{3.5}} - \frac{1}{\lambda_0^{3.5}}
   $$

---

### ⚙️ Widget Workflow Summary

1. **Input:** Read glass types and sliders:  
   $ \lambda_1,\ \lambda_2,\ \lambda_0,\ f_0 $

2. **Achromat ratio:**  
   Compute $ r = K_2/K_1 $ via Eq. 4.

3. **Shape factors:**  
   Solve $ K_1, K_2 $ from Eq. 5 using the focal length constraint.

4. **Power expansion:**  
   Assemble $ \Phi_0, \alpha, \beta $ via Eq. 3 and convert to $ A, B, C $ using Eq. 6.

5. **Chromatic shift curve:**  
   Compute $ \Delta z(\lambda) $ from both the direct formula and analytic form.  
   Both expressions are plotted; matching results verify correctness.

6. **Export:**  
   Sampled $[ \lambda, \Delta z ]$ every 10 nm is saved to the global array `CHLdata_global`.

---


In [ ]:
# -------- 0. λ-grid -------------------------------------------------------
lam_max = 700                     # nm, max wavelength
lam_f = np.arange(400, lam_max, 1)          # nm, master grid
CHLdata_global = None
out1 = Output()

# -------- 1. Conrady coeffs (nm domain) -----------------------------------

crown_prefix = ("BK", "BAK", "FK", "LAK")
crown_glasses = [g for g in glass_conrady if any(p in g for p in crown_prefix)]
flint_glasses = [g for g in glass_conrady if g not in crown_glasses]

# -------- 2. Widgets ------------------------------------------------------
g1_dd = Dropdown(options=crown_glasses, value="N-BAK4", description="Glass 1")
g2_dd = Dropdown(options=flint_glasses, value="SF10",   description="Glass 2")

f0_sl = FloatSlider(min=50, max=200, step=10, value=100,
                    description="f₀ [mm]", continuous_update=False,
                    readout_format=".0f", layout={"width": "380px"})

lam1_sl = FloatSlider(min=400, max=lam_max, step=1, value=486.1,
                      description="λ₁ [nm]", continuous_update=False,
                      layout={"width": "380px"})
lam2_sl = FloatSlider(min=400, max=lam_max, step=1, value=656.3,
                      description="λ₂ [nm]", continuous_update=False,
                      layout={"width": "380px"})
lam0_sl = FloatSlider(min=400, max=lam_max, step=1, value=587.3,
                      description="λ₀ [nm]", continuous_update=False,
                      layout={"width": "380px"})

# -------- 3. Helpers ------------------------------------------------------
def chrom_shift(lam_nm, A, B, C):
    """Δz(λ) = A + B/λ + C/λ³·⁵, λ in nm, result in µm."""
    lam_nm = np.asarray(lam_nm, dtype=np.float64)
    return A + B/lam_nm + C/lam_nm**3.5

# -------- 4. Callback -----------------------------------------------------
def update1(g1, g2, f0, λ1, λ2, λ0):
    global CHLdata_global
    with out1:
        clear_output(wait=True)

        if abs(λ1 - λ2) < 1e-6:
            print("λ₁ and λ₂ must differ.")
            return

        # design power in mm⁻¹
        Phi0_req = 1.0 / f0

        # --- Conrady coeffs ---
        a1, b1, c1 = (glass_conrady[g1][k] for k in ("a", "b", "c"))
        a2, b2, c2 = (glass_conrady[g2][k] for k in ("a", "b", "c"))

        # --- Achromat condition: solve r ---
        Δν    = 1/λ1     - 1/λ2
        Δν35  = 1/λ1**3.5 - 1/λ2**3.5
        num   = b1*Δν   + c1*Δν35
        den   = b2*Δν   + c2*Δν35
        if abs(den) < 1e-12:
            print("Achromat condition degenerate")
            return
        r = -num / den

        # --- Design power constraint: solve K1, then K2 ---
        denom = (a1 + r*a2) + (b1 + r*b2)/λ0 + (c1 + r*c2)/λ0**3.5
        if abs(denom) < 1e-12:
            print("Division by zero in Φ_total expression")
            return
        K1 = Phi0_req / denom
        K2 = r * K1

        # --- Total power coefficients ---
        a_tot = a1*K1 + a2*K2
        b_tot = b1*K1 + b2*K2
        c_tot = c1*K1 + c2*K2

        # precompute Φ(λ) and Φ(λ₀)
        def Phi_tot(lam): return a_tot + b_tot/lam + c_tot/lam**3.5
        Φ0 = Phi_tot(λ0)

        # --- Direct Δz (always refocused at λ₀) ---
        mm_to_um = 1e3
        dz_direct = mm_to_um * (1 / Phi_tot(lam_f) - 1 / Φ0)

        # --- Analytic Δz via Conrady (always zero at λ₀) ---
        A = mm_to_um / Φ0
        B = -mm_to_um * b_tot / Φ0**2
        C = -mm_to_um * c_tot / Φ0**2
        dz_analytic = B * (1/lam_f - 1/λ0) + C * (1/lam_f**3.5 - 1/λ0**3.5)

        # --- Sample for export ---
        lam_sample = np.arange(400, lam_max, 10)
        dz_sample = mm_to_um * (1 / Phi_tot(lam_sample) - 1 / Φ0)
        CHLdata_global = np.column_stack((lam_sample, dz_sample))

        # --- Plotting ---
        fig, (ax_z, ax_cmp) = plt.subplots(1, 2, figsize=(12, 5), constrained_layout=True)
        ax_z.plot(lam_f, dz_direct,    "r", lw=2, label="Δz (direct)")
        ax_z.plot(lam_f, dz_analytic, "--k", lw=0.8, label="Δz (analytic)")
        ax_z.axvline(λ1, color="k", ls=":", lw=0.5)
        ax_z.axvline(λ2, color="k", ls=":", lw=0.5)
        ax_z.axvline(λ0, color="m", ls="--", lw=0.5, label="λ₀")
        ax_z.set(xlabel="λ [nm]", ylabel="Δz [µm]", xlim=(400, lam_max))
        ax_z.grid(lw=0.3)
        ax_z.legend(fontsize=8)

        ax_cmp.plot(lam_f, B/lam_f,       "g", label="B / λ")
        ax_cmp.plot(lam_f, C/lam_f**3.5,  "b", label="C / λ³·⁵")
        ax_cmp.set(xlabel="λ [nm]", ylabel="Component [µm]", xlim=(400, lam_max))
        ax_cmp.grid(lw=0.3)
        ax_cmp.legend(fontsize=8)

        fig.suptitle(
            f"{g1} + {g2}   f₀ = {f0:.1f} mm   "
            f"K₁ = {K1:+.4f}, K₂ = {K2:+.4f} (r = {r:+.4f})  (refocus @ {λ0:.0f} nm)",
            fontsize=12)
        plt.show()
        plt.close(fig)

        # --- Console read-out ---
        print(f"Check: design Φ(λ₀) = {Φ0:+.4f} mm⁻¹  (should = {Phi0_req:+.4f})")
        print(f"K₁ = {K1:+.4f}    K₂ = {K2:+.4f}")
        print(f"A = {A:+.2f} µm   B = {B:+.2e} µm·nm   C = {C:+.2e} µm·nm³·⁵")
        print("CHLdata_global →", CHLdata_global.shape)

# -------- 5. Launch -------------------------------------------------------
interact(update1,
         g1=g1_dd, g2=g2_dd, f0=f0_sl,
         λ1=lam1_sl, λ2=lam2_sl, λ0=lam0_sl)
display(out1)


interactive(children=(Dropdown(description='Glass\u202f1', index=4, options=('N-BK7', 'N-BK10', 'N-FK5', 'N-BA…

Output()

In [ ]:
# -------------------------------------------------------------------------
# 1. λ‑grid ---------------------------------------------------------------
lam_max = 1000                         # nm
lam_f    = np.arange(400, lam_max, 1)  # nm master grid
CHLdata_global = None
out2 = Output()

# -------------------------------------------------------------------------
# 2. Widget setup (unchanged) ---------------------------------------------
crown_prefix = ("BK", "BAK", "FK", "LAK")
crown_glasses = [g for g in glass_conrady if any(p in g for p in crown_prefix)]
flint_glasses = [g for g in glass_conrady if g not in crown_glasses]

g1_dd = Dropdown(options=crown_glasses, value="N-BAK4", description="Glass 1")
g2_dd = Dropdown(options=flint_glasses, value="SF10",   description="Glass 2")

f0_sl  = FloatSlider(min=50,  max=200, step=10, value=100,
                     description="f₀ [mm]",  continuous_update=False,
                     readout_format=".0f", layout={"width": "380px"})
lam1_sl = FloatSlider(min=400, max=lam_max, step=1, value=486.1,
                      description="λ₁ [nm]", continuous_update=False,
                      layout={"width": "380px"})
lam2_sl = FloatSlider(min=400, max=lam_max, step=1, value=656.3,
                      description="λ₂ [nm]", continuous_update=False,
                      layout={"width": "380px"})
lam0_sl = FloatSlider(min=400, max=lam_max, step=1, value=587.3,
                      description="λ₀ [nm]", continuous_update=False,
                      layout={"width": "380px"})

# -------------------------------------------------------------------------
# 3. Helper functions ------------------------------------------------------

def n_sellmeier(coeffs, lam_nm):
    """Sellmeier index; lam_nm is scalar or ndarray."""
    if isinstance(coeffs, (list, tuple)):
        B1, B2, B3, C1, C2, C3 = coeffs
    else:
        d = {k.lower(): v for k, v in coeffs.items()}
        B1, B2, B3 = d["b1"], d["b2"], d["b3"]
        C1, C2, C3 = d["c1"], d["c2"], d["c3"]
    lam_um = lam_nm * 1e-3
    lam2   = lam_um**2
    n2 = 1 + B1*lam2/(lam2 - C1) + B2*lam2/(lam2 - C2) + B3*lam2/(lam2 - C3)
    return np.sqrt(n2)

# --- NEW -----------------------------------------------------------------
def n_acf(coeffs, lam_nm):
    """ACF dispersion: n = 1 + a + b/λ² + c/λ⁴ + d·λ²  (λ in nm)."""
    d = {k.lower(): v for k, v in coeffs.items()}
    a, b, c, dcoef = d["a"], d["b"], d["c"], d["d"]
    lam2 = lam_nm**2
    return 1 + a + b/lam2 + c/lam2**2 + dcoef*lam2

# -------------------------------------------------------------------------
# 4. Unified callback ------------------------------------------------------
def update2(g1, g2, f0, λ1, λ2, λ0):
    global CHLdata_global
    with out2:
        clear_output(wait=True)

        if abs(λ1 - λ2) < 1e-6:
            print("λ₁ and λ₂ must differ.")
            return

        Phi0_req = 1.0 / f0         # target power [mm⁻¹]

        # -------- Conrady -------------------------------------------------
        a1, b1, c1 = (glass_conrady[g1][k] for k in ("a", "b", "c"))
        a2, b2, c2 = (glass_conrady[g2][k] for k in ("a", "b", "c"))

        Δν   = 1/λ1 - 1/λ2
        Δν35 = 1/λ1**3.5 - 1/λ2**3.5
        r_c  = -(b1*Δν + c1*Δν35) / (b2*Δν + c2*Δν35)

        denom_c = (a1+r_c*a2) + (b1+r_c*b2)/λ0 + (c1+r_c*c2)/λ0**3.5
        K1 = Phi0_req / denom_c
        K2 = r_c * K1

        a_tot = a1*K1 + a2*K2
        b_tot = b1*K1 + b2*K2
        c_tot = c1*K1 + c2*K2
        def Phi_tot(lam):
            return a_tot + b_tot/lam + c_tot/lam**3.5
        Φ0c = Phi_tot(λ0)

        mm_to_um = 1e3
        dz_conrady      = mm_to_um * (1/Phi_tot(lam_f) - 1/Φ0c)
        B = -mm_to_um * b_tot / Φ0c**2
        C = -mm_to_um * c_tot / Φ0c**2
        dz_conrady_fit  = B*(1/lam_f - 1/λ0) + C*(1/lam_f**3.5 - 1/λ0**3.5)

        # -------- Sellmeier ----------------------------------------------
        s1, s2 = glass_sellmeier[g1], glass_sellmeier[g2]
        n1_l1, n1_l2, n1_l0 = [n_sellmeier(s1, l) for l in (λ1, λ2, λ0)]
        n2_l1, n2_l2, n2_l0 = [n_sellmeier(s2, l) for l in (λ1, λ2, λ0)]

        Δn1 = n1_l1 - n1_l2
        Δn2 = n2_l1 - n2_l2
        r_s = -Δn1 / Δn2

        denom_s = (n1_l0 - 1) + r_s*(n2_l0 - 1)
        Φ1 = Phi0_req / denom_s
        Φ2 = r_s * Φ1

        Φ_grid = Φ1*(n_sellmeier(s1, lam_f) - 1) + \
                 Φ2*(n_sellmeier(s2, lam_f) - 1)
        Φ0s = Φ1*(n1_l0 - 1) + Φ2*(n2_l0 - 1)
        dz_sellmeier = mm_to_um * (1/Φ_grid - 1/Φ0s)

        # Keep CHL output based on Sellmeier only (unchanged behaviour)
        lam_sample = np.arange(400, lam_max, 10)
        Φ_s = Φ1*(n_sellmeier(s1, lam_sample)-1) + \
              Φ2*(n_sellmeier(s2, lam_sample)-1)
        dz_sample = mm_to_um*(1/Φ_s - 1/Φ0s)
        CHLdata_global = np.column_stack((lam_sample, dz_sample))

        # -------- glass_acf ---------------------------------------------------
        b1c, b2c = glass_acf[g1], glass_acf[g2]
        nb1_l1, nb1_l2, nb1_l0 = [n_acf(b1c, l) for l in (λ1, λ2, λ0)]
        nb2_l1, nb2_l2, nb2_l0 = [n_acf(b2c, l) for l in (λ1, λ2, λ0)]

        Δnb1 = nb1_l1 - nb1_l2
        Δnb2 = nb2_l1 - nb2_l2
        r_b  = -Δnb1 / Δnb2

        denom_b = (nb1_l0 - 1) + r_b*(nb2_l0 - 1)
        Φ1b = Phi0_req / denom_b
        Φ2b = r_b * Φ1b

        Φ_grid_b = Φ1b*(n_acf(b1c, lam_f) - 1) + \
                   Φ2b*(n_acf(b2c, lam_f) - 1)
        Φ0b = Φ1b*(nb1_l0 - 1) + Φ2b*(nb2_l0 - 1)
        dz_acf = mm_to_um * (1/Φ_grid_b - 1/Φ0b)

        # -------- Plotting ------------------------------------------------
        fig, (ax_z, ax_cmp) = plt.subplots(1, 2, figsize=(12, 5),
                                           constrained_layout=True)
        ax_z.plot(lam_f, dz_conrady,     "r",  lw=2, label="Conrady direct")
        ax_z.plot(lam_f, dz_conrady_fit, "--k", lw=0.8,
                  label="Conrady analytic")
        ax_z.plot(lam_f, dz_sellmeier,   "b",  lw=2, label="Sellmeier direct")
        ax_z.plot(lam_f, dz_acf,       "g",  lw=2, label="ACF direct")

        for l in (λ1, λ2, λ0):
            ax_z.axvline(l, color="k", ls=":", lw=0.5)
        ax_z.set(xlabel="λ [nm]", ylabel="Δz [µm]", xlim=(400, lam_max))
        ax_z.grid(lw=0.3)
        ax_z.legend(fontsize=8)

        ax_cmp.plot(lam_f, B/lam_f,      "g", label="B / λ")
        ax_cmp.plot(lam_f, C/lam_f**3.5, "b", label="C / λ³·⁵")
        ax_cmp.set(xlabel="λ [nm]", ylabel="Component [µm]", xlim=(400, lam_max))
        ax_cmp.grid(lw=0.3)
        ax_cmp.legend(fontsize=8)

        fig.suptitle(
            f"{g1} + {g2}   f₀={f0:.1f} mm   "
            f"K₁={K1:+.4f} K₂={K2:+.4f}   "
            f"Φ₁={Φ1:+.4f} Φ₂={Φ2:+.4f}   "
            f"Φ₁ʙ={Φ1b:+.4f} Φ₂ʙ={Φ2b:+.4f}   (λ₀={λ0:.0f} nm)",
            fontsize=12)
        plt.show()

        # -------- Console summary ---------------------------------------
        print(f"Conrady  : Φ(λ₀)={Φ0c:+.4f} mm⁻¹")
        print(f"           K₁={K1:+.4f}  K₂={K2:+.4f}  r={r_c:+.4f}")
        print(f"Sellmeier: Φ(λ₀)={Φ0s:+.4f} mm⁻¹")
        print(f"           Φ₁={Φ1:+.4f}  Φ₂={Φ2:+.4f}  r={r_s:+.4f}")
        print(f"ACF      : Φ(λ₀)={Φ0b:+.4f} mm⁻¹")
        print(f"           Φ₁ʙ={Φ1b:+.4f} Φ₂ʙ={Φ2b:+.4f} r={r_b:+.4f}")
        print("CHLdata_global →", CHLdata_global.shape)


# -------------------------------------------------------------------------
# 5. Wire up the interactive controls -------------------------------------
interact(update2,
         g1=g1_dd, g2=g2_dd, f0=f0_sl,
         λ1=lam1_sl, λ2=lam2_sl, λ0=lam0_sl)
display(out2)


interactive(children=(Dropdown(description='Glass\u202f1', index=4, options=('N-BK7', 'N-BK10', 'N-FK5', 'N-BA…

Output()

### Defocus-Sweep Post-Analysis  

This cell evaluates how **lateral colour blur** varies when the image
plane is stepped through a ± defocus range and reports:

* the **maximum blur width** (worst case),
* the **mean blur width** between the first and last zero crossing
  (i.e. across the in-focus zone),
* the two **zero-crossing positions** themselves.

---

#### Constants & Units  

| Name | Default | Units | Meaning |
|------|---------|-------|---------|
| `K` | 2.2 | – | lens f-number $\,K=f/\!D$ |
| `F_VALUE` | 8.0 | – | exposure-curve factor used elsewhere |
| `GAMMA_VALUE` | 1.0 | – | gamma correction factor |
| `TOL` | 0.15 | – | colour‐difference threshold for “acceptable” blur |
| `XRANGE` | 400 | $\mu\text{m}$ | half-width of the lateral-colour evaluation window |
| `defocusrange` | 1500 | $\mu\text{m}$ | half range of the longitudinal sweep |
| `defocus_step` | 10 | $\mu\text{m}$ | step size of the sweep |
| `threshold` | $10^{-6}$ | – | numerical zero for width crossing |

The defocus sweep is therefore  
$z\in[-1500,+1500]\,\mu\text{m}$ in $10\,\mu\text{m}$ increments.

---

#### Workflow  

1. **Generate defocus grid**  
   `z_vals = np.arange(-defocusrange, …, defocus_step)`

2. **Evaluate colour blur**  
   `widths[i] = fringe_width(z_i, CHLdata)`  
   (function returns the lateral colour *width* in µm at each defocus plane,
   using the sampled longitudinal-chromatic data `CHLdata_global`).

3. **Locate zero crossings**  
   Linear interpolation between successive samples where  
   $\text{width}(z)$ changes sign with respect to `threshold`.

4. **Statistics**  
   * `widths.max()` — worst blur in the sweep  
   * `mean_valid`   — mean width between first and last zero  
   * positions of the two zeros, `z0_left` and `z0_right`

5. **Diagnostic message**  
   If fewer than two zero crossings are found, the code suggests increasing
   `defocusrange`.


In [6]:
# ---------------------------------------------------------------------
# Constants (edit here, everything else updates automatically)
# ---------------------------------------------------------------------
K: float            = 2.2   # f‑number
F_VALUE: float      = 8.0   # default exposure‑curve factor
GAMMA_VALUE: float  = 1.0   # default gamma

TOL: float          = 0.15  # colour‑difference tolerance

XRANGE      = 400        # x window half width (µm)

defocusrange: int   = 1000  # ± defocus sweep range (µm)
defocus_step: int   = 10    # defocus sampling step (µm)

z_vals = np.arange(-defocusrange,
                   defocusrange + defocus_step,
                   defocus_step, dtype=float)


In [9]:
threshold = 1e-6

widths = np.array([fringe_width(z, CHLdata=CHLdata_global[:, 1]) for z in z_vals])


zero_crossings = []
for i in range(len(widths) - 1):
    w0, w1 = widths[i], widths[i + 1]
    if w0 <= threshold and w1 > threshold:
        z0 = np.interp(0.0, [w0, w1], [z_vals[i], z_vals[i + 1]])
        zero_crossings.append(z0)
    elif w0 > threshold and w1 <= threshold:
        z0 = np.interp(0.0, [w0, w1], [z_vals[i], z_vals[i + 1]])
        zero_crossings.append(z0)

if len(zero_crossings) >= 2:
    z0_left, z0_right = zero_crossings[0], zero_crossings[-1]

    mask = (z_vals >= z0_left) & (z_vals <= z0_right)
    mean_valid = widths[mask].mean()

    print(f"Max  width: {widths.max():.2f}")
    print(f"Mean width (between first & last zero): {mean_valid:.2f}")
    print(f"First zero at  z ≈ {z0_left:.3f}    " f"Last zero at  z ≈ {z0_right:.3f}")
    print("To make the computation faster, use a narrower defocus range.")

else:
    print("Please enlarge the defocus range to find zero crossings.")


TypeError: fringe_width() got an unexpected keyword argument 'CHLdata'